In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [2]:
# Loading and displaying the dataset
file_path = '/Users/kyla/Desktop/paml-lyb/ML-in-Real-Estate/data/New York, NY_sold_past365days.csv'
housing_data = pd.read_csv(file_path)
housing_data.head()
housing_data.info()
housing_data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   status           10000 non-null  object 
 1   style            10000 non-null  object 
 2   street           9979 non-null   object 
 3   city             10000 non-null  object 
 4   zip_code         9999 non-null   float64
 5   beds             8703 non-null   float64
 6   full_baths       8464 non-null   float64
 7   half_baths       2749 non-null   float64
 8   sqft             6904 non-null   float64
 9   year_built       8563 non-null   float64
 10  days_on_mls      6310 non-null   float64
 11  list_price       8875 non-null   float64
 12  sold_price       10000 non-null  int64  
 13  assessed_value   5975 non-null   float64
 14  estimated_value  6933 non-null   float64
 15  lot_sqft         5223 non-null   float64
 16  price_per_sqft   6904 non-null   float64
 17  latitude     

,zip_code,beds,full_baths,half_baths,sqft,year_built,days_on_mls,list_price,sold_price,assessed_value,estimated_value,lot_sqft,price_per_sqft,latitude,longitude,stories
count,9999.000000,8703.000000,8464.000000,2749.000000,6904.000000,8563.000000,6310.000000,8.875000e+03,1.000000e+04,5.975000e+03,6.933000e+03,5.223000e+03,6904.000000,9324.000000,9324.000000,7205.000000
mean,10801.272027,2.905550,1.749646,1.086213,1780.949160,1957.922691,170.256260,1.104134e+06,1.096758e+06,1.087258e+05,1.023309e+06,7.271305e+03,717.417005,40.682508,-73.962871,6.401249
std,563.003598,1.798272,1.006683,0.337259,5302.020161,34.218667,118.808031,4.693596e+06,5.870682e+06,1.421254e+06,1.169288e+06,5.440150e+04,3338.021665,0.090531,0.131160,11.116396
min,10001.000000,0.000000,1.000000,1.000000,300.000000,1601.000000,0.000000,2.500000e+03,6.200000e+02,6.000000e+01,2.065800e+04,1.530000e+02,1.000000,40.498702,-77.874664,0.000000
25%,10305.000000,2.000000,1.000000,1.000000,975.000000,1930.000000,100.000000,4.999000e+05,4.800000e+05,3.612000e+04,6.070000e+05,2.000000e+03,375.000000,40.610983,-74.024077,2.000000
50%,11204.000000,3.000000,1.000000,1.000000,1381.500000,1955.000000,143.000000,7.500000e+05,7.399995e+05,4.842000e+04,8.019280e+05,2.640000e+03,495.000000,40.686435,-73.958288,2.000000
75%,11356.000000,4.000000,2.000000,1.000000,2054.250000,1985.000000,208.000000,1.150000e+06,1.115000e+06,7.113000e+04,1.127741e+06,4.000000e+03,746.000000,40.757561,-73.876367,4.000000
max,14481.000000,22.000000,13.000000,10.000000,260000.000000,2024.000000,1510.000000,3.995000e+08,5.650000e+08,1.040778e+08,3.808595e+07,2.744280e+06,274538.000000,43.004954,-73.701469,96.000000


In [3]:
# Handling missing data
missing_data = housing_data.isnull().sum()
missing_data_percentage = (missing_data / len(housing_data)) * 100
missing_data_summary = pd.DataFrame({'Missing Values': missing_data, 'Percentage': missing_data_percentage})
missing_data_summary.sort_values(by='Percentage', ascending=False)

,Missing Values,Percentage
half_baths,7251,72.51
lot_sqft,4777,47.77
assessed_value,4025,40.25
days_on_mls,3690,36.90
price_per_sqft,3096,30.96
sqft,3096,30.96
estimated_value,3067,30.67
stories,2795,27.95
full_baths,1536,15.36
year_built,1437,14.37


In [4]:
# Data cleaning steps
housing_data['half_baths'].fillna(0, inplace=True)
housing_data.drop(columns=['days_on_mls', 'assessed_value', 'estimated_value', 'lot_sqft', 'stories'], inplace=True)
median_sqft = housing_data['sqft'].median()
housing_data['sqft'].fillna(median_sqft, inplace=True)
housing_data.loc[housing_data['price_per_sqft'].isnull(), 'price_per_sqft'] = housing_data['sold_price'] / housing_data['sqft']

median_values = {
    'full_baths': housing_data['full_baths'].median(),
    'beds': housing_data['beds'].median(),
    'year_built': housing_data['year_built'].median(),
    'list_price': housing_data['list_price'].median()
}
housing_data.fillna(median_values, inplace=True)

for zip_code in housing_data['zip_code'].unique():
    lat_median = housing_data.loc[housing_data['zip_code'] == zip_code, 'latitude'].median()
    long_median = housing_data.loc[housing_data['zip_code'] == zip_code, 'longitude'].median()
    housing_data.loc[(housing_data['zip_code'] == zip_code) & (housing_data['latitude'].isnull()), 'latitude'] = lat_median
    housing_data.loc[(housing_data['zip_code'] == zip_code) & (housing_data['longitude'].isnull()), 'longitude'] = long_median

/var/folders/84/407g7szd0h91zh0w0ftr4brh0000gn/T/ipykernel_21006/1481647926.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  housing_data['half_baths'].fillna(0, inplace=True)
/var/folders/84/407g7szd0h91zh0w0ftr4brh0000gn/T/ipykernel_21006/1481647926.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values al

In [5]:
# Further data cleaning
housing_data['zip_code'].fillna(housing_data['zip_code'].mode()[0], inplace=True)
housing_data['county'].fillna(housing_data['county'].mode()[0], inplace=True)
housing_data.dropna(subset=['latitude', 'longitude'], inplace=True)

/var/folders/84/407g7szd0h91zh0w0ftr4brh0000gn/T/ipykernel_21006/3029240019.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  housing_data['zip_code'].fillna(housing_data['zip_code'].mode()[0], inplace=True)
/var/folders/84/407g7szd0h91zh0w0ftr4brh0000gn/T/ipykernel_21006/3029240019.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on

In [6]:
# Type conversions and filtering
housing_data['zip_code'] = housing_data['zip_code'].astype(int)
housing_data = housing_data[(housing_data['latitude'].between(40.5, 41.5)) & (housing_data['longitude'].between(-74.5, -73.5))]

In [7]:
# Feature engineering
current_year = 2024
housing_data['house_age'] = current_year - housing_data['year_built']
encoder = OneHotEncoder(sparse_output=False, drop='first')
style_encoded = encoder.fit_transform(housing_data[['style']])
style_encoded_df = pd.DataFrame(style_encoded, columns=encoder.get_feature_names_out())
housing_data = pd.concat([housing_data.reset_index(drop=True), style_encoded_df], axis=1)

In [8]:
# Adding new features
housing_data['total_baths'] = housing_data['full_baths'] + 0.5 * housing_data['half_baths']
housing_data['beds_times_sqft'] = housing_data['beds'] * housing_data['sqft']
housing_data['total_baths_times_sqft'] = housing_data['total_baths'] * housing_data['sqft']

housing_data.shape

(9981, 29)

In [9]:
housing_data.head()

,status,style,street,city,zip_code,beds,full_baths,half_baths,sqft,year_built,...,style_COOP,style_LAND,style_MOBILE,style_MULTI_FAMILY,style_OTHER,style_SINGLE_FAMILY,style_TOWNHOMES,total_baths,beds_times_sqft,total_baths_times_sqft
0,SOLD,LAND,340 Manor Rd,Staten Island,10314,3.0,1.0,0.0,1381.5,1955.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4144.5,1381.5
1,SOLD,SINGLE_FAMILY,30 Hillview Ln,Staten Island,10304,4.0,2.0,0.0,3400.0,1985.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,13600.0,6800.0
2,SOLD,SINGLE_FAMILY,80 Longview Rd,Staten Island,10304,3.0,1.0,1.0,2205.0,1987.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.5,6615.0,3307.5
3,SOLD,SINGLE_FAMILY,78 Hamden Ave,Staten Island,10306,2.0,1.0,1.0,1300.0,1920.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.5,2600.0,1950.0
4,SOLD,SINGLE_FAMILY,395 Little Clove Rd,Staten Island,10301,2.0,2.0,0.0,900.0,1955.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1800.0,1800.0
